In [ ]:
from __future__ import absolute_import, division, print_function
import json
from collections import namedtuple

In [ ]:
def time_dependent_value(year=2016, values=None):
    """
    :param year: Year for which we are looking for parameter
    :param values: dictionary in the form {"2016": 1.0, "2017": 2.0}
    :return: determined value, first year for which its valid (for calculating improvements from a known point)

    """

    values = values or {}
    value = None
    lastYear = None
    for deltaYear in sorted(values.keys()):
        if int(year) >= int(deltaYear):
            lastYear = int(deltaYear)
            value = values[deltaYear]

    return value, lastYear

In [ ]:
def interpolate_value(ramp, year):
    """
    Takes a dictionary of the form
    {"2016": x,
     "2020": y,
     ...
     }

     and returns x for year=2016, y for year=2020, and an interpolated value for 2017, 2018, 2019
    """

    pastYear = 0
    futureYear = 3000
    value = None
    for otherType in sorted(ramp):
        otherYear = int(otherType)
        if year == otherYear:  # We found the exact value
            value = ramp[otherType]
            break
        if year - otherYear < year - pastYear and year > otherYear:
            pastYear = otherYear
        if otherYear > year:
            futureYear = otherYear
            break

    if value is None:  # We didn't get an exact value, interpolate between two values
        value = (ramp[str(pastYear)] + (year - pastYear) *
                 (ramp[str(futureYear)] - ramp[str(pastYear)]) / (futureYear - pastYear))

    return value

In [ ]:
SECONDS_PER_YEAR = 365.25 * 24 * 3600
kilo = 1000
mega = 1000 * kilo
giga = 1000 * mega
tera = 1000 * giga
peta = 1000 * tera
seconds_per_year = 86400 * 365
seconds_per_month = 86400 * 30

In [ ]:
def updateDict(target,changes):
    for k,v in changes.items():
        if k in target and isinstance(target[k],dict):
            updateDict(target[k],v)
        else:
            target[k]=v

In [ ]:

def configure(modelName):
    import json
    modelNames = ['json/BaseModel.json', 'json/RealisticModel.json']

    if isinstance(modelName, basestring):
        modelNames.append(modelName)
    elif isinstance(modelName, list):
        modelNames.extend(modelName)

    model = {}
    for modelName in modelNames:
        #print(modelName)
        with open(modelName, 'r') as modelFile:
            modelChanges = json.load(modelFile)
            updateDict(model,modelChanges)
#            model.update(modelChanges)

    return model

In [ ]:
def in_shutdown(model, year):
    """
    :param model: The configuration dictionary
    :param year: check if this year is in a shutdown period
    :return: boolean for in shutdown, integer for last year not in shutdown
    """

    inShutdown = year in model['shutdown_years']

    while year in model['shutdown_years']:
        year -= 1

    return inShutdown, year

In [ ]:
def run_model(model, year, data_type='data'):
    """
    :param model: The configuration dictionary
    :param year: The year the model is being queried for
    :param data_type: The type of data (MC or data)
    :return: data events, in_shutdown
    """

    RunModel = namedtuple('RunModel', 'events, in_shutdown')

    inShutdown, lastRunningYear = in_shutdown(model, year)
    events = 0
    if not inShutdown:
        triggerRate, basisYear = time_dependent_value(year, model['trigger_rate'])
        liveFraction, basisYear = time_dependent_value(year, model['live_fraction'])
        events = SECONDS_PER_YEAR * liveFraction * triggerRate
    if data_type == 'mc':
        events *= model['mc_event_factor']
    return RunModel(events, inShutdown)

In [ ]:
def mc_event_model(model, year):
    """
    Given the various types of MC and their fraction compared to data in mc_evolution,
    for a the queried year, return the number of events needed to be simulated of each
    "MC year" in that calendar year.

    :param model: The configuration dictionary
    :param year: The year the model is being queried for
    :return: dictionary of {year1: events, year2: events} of types of events needed to be simualted
    """

    mcEvolution = model['mc_evolution']
    mcEvents = {}
    for mcType, ramp in mcEvolution.items():
        mcYear = int(mcType)

        # First figure out what to base the number of MC events
        currEvents = run_model(model, year).events
        if in_shutdown(model, year)[0]:
            lastYear = in_shutdown(model, year)[1]
            lastEvents = run_model(model, lastYear).events
        else:
            lastEvents = 0

        if mcYear > year:
            futureEvents = run_model(model, mcYear).events
        else:
            futureEvents = 0
        dataEvents = max(currEvents, lastEvents, futureEvents)

        # TODO: Replace this bit of code with interpolate_value from utils.py
        pastYear = 0
        futureYear = 3000
        mc_fraction = None
        for otherType in sorted(ramp):
            otherYear = int(otherType)
            if year == otherYear:  # We found the exact value
                mc_fraction = ramp[otherType]
                break
            if year - otherYear < year - pastYear and year > otherYear:
                pastYear = otherYear
            if otherYear > year:
                futureYear = otherYear
                break

        if mc_fraction is None:  # We didn't get an exact value, interpolate between two values
            mc_fraction = (ramp[str(pastYear)] + (year - pastYear) *
                           (ramp[str(futureYear)] - ramp[str(pastYear)]) / (futureYear - pastYear))

        mcEvents[mcType] = mc_fraction * dataEvents

    return mcEvents

In [ ]:
def performance_by_year(model, year, tier, data_type=None, kind=None):
    """
    Return various performance metrics based on the year under consideration
    (allows for step and continuous variations)

    :param model: The model parameters
    :param year: The year in which processing is done
    :param tier: Data tier produced
    :param data_type: data or mc
    :param kind: The year flavor of MC or data. May differ from actual running year

    :return:  tuple of cpu time (HS06 * s) and data size
    """

    # If we don't specify flavors, assume we are talking about the current year

    # TODO:  Big old hack for now because we don't have "kind" for data
    if not kind:
        # print year
        kind = str(year)
    #print(year)
    if kind not in ['2017', '2026']:
        if int(kind) >= 2025:
            kind = '2026'
        else:
#            if int(kind) >= 2020:
#                kind = '2021'
#            else:
            kind = '2017'

# it gets worse - there is run 3
    if kind == '2017' and year > 2020:
        kind='2021'


    kind = str(kind)

    try:
        for modelYear in sorted(model['tier_sizes'][tier].keys()):
            if int(kind) >= int(modelYear):
                sizePerEvent = model['tier_sizes'][tier][modelYear]
    except KeyError:  # Storage model does not know this tier
        sizePerEvent = None

    try:
        # Look up the normalized processing time
        for modelYear in sorted(model['cpu_time'][data_type][tier].keys()):
            if int(kind) >= int(modelYear):
                cpuPerEvent = model['cpu_time'][data_type][tier][modelYear]

        # Apply the year by year correction
        improvement_factor = 1.0
        ramp = model['improvement_factors']['software_by_kind'][kind]
        for improve_year in range(int(model['start_year']), int(year) + 1):
            year_factor = interpolate_value(ramp, improve_year)
            improvement_factor *= year_factor

        cpuPerEvent = cpuPerEvent / improvement_factor
    except KeyError:  # CPU model does not know this tier
        cpuPerEvent = None

    return cpuPerEvent, sizePerEvent
